## Anthropic Agent Class

In [38]:
from enum import Enum, auto
from anthropic import Anthropic
from abc import ABC, abstractmethod
from typing import List, Dict, Any, Optional
import os
from smart_team.agents.agent_functions import get_weather
from dynamic_agent.utils.function_schema import create_function_schema, SchemaFormat

class AgentState(Enum):
    """States that an agent can be in"""
    READY = auto()           # Ready to receive new input
    PROCESSING = auto()      # Processing a task
    COMPLETED = auto()       # Task completed, ready to transfer
    TRANSFERRING = auto()    # In process of transferring


In [39]:
from abc import ABC, abstractmethod
from typing import List, Dict, Any, Optional
from pydantic import BaseModel, Field
from dataclasses import dataclass

@dataclass
class AgentResponse:
    """Response from an agent"""
    text: str | None = None
    function_calls: list[dict[str, any]] | None = None


class BaseAgent(ABC):
    """Base class for all agents"""

    def __init__(self, name: str, instructions: str, functions: List = None, **kwargs):
        """Initialize the agent"""
        self.is_ochestrator: bool = False
        self.name = name
        self.instructions = instructions
        self.functions = functions or []
        self.state = AgentState.READY
        self._init_client(**kwargs)

    @abstractmethod
    def _init_client(self, **kwargs):
        """Initialize the client - to be implemented by specific agents"""
        pass

    @abstractmethod
    def send_message(self, message: str, is_function_result: bool = False) -> AgentResponse:
        """Send a message to the agent and get a response"""
        pass


class AnthropicAgent(BaseAgent):
    def _init_client(self, **kwargs):
        self.model = kwargs.get("model", "claude-3-5-sonnet-20241022")
        api_key = kwargs.get("api_key")
        if not api_key:
            raise ValueError("api_key is required for AnthropicAgent")
        self.client = Anthropic(api_key=api_key)
        self.agent_memory = []
    
        
    def _get_tool_schemas(self) -> List[Dict]:
        """Convert functions to Anthropic's tool format"""
        return [
            create_function_schema(func, SchemaFormat.ANTHROPIC)
            for func in self.functions
        ]
        
    def send_message(self, messages:Dict) -> AgentResponse:
        # Get Agent Tool Schema
        tools = self._get_tool_schemas() if self.functions else []
        
        # Get response from Claude
        response = self.client.messages.create(
            model=self.model,
            messages=messages,
            max_tokens=8192,
            tools=tools,
        )
        
        # Get the response texts and the function calls seperately
        text_parts = []
        result = AgentResponse()
        result.function_calls = []
        for block in response.content:
            if block.type == "text":
                text_parts.append(block.text)
        
            elif block.type == "tool_use":
                # Check if this exact function call with these parameters has been made before
                func_call = {
                    "name": block.name,
                    "parameters": block.input,
                }
                result.function_calls.append(func_call)

        result.text = " ".join(text_parts) if text_parts else ""
        return result
                
    def get_agent_memory(self):
        pass

In [64]:
def transfer_to_weather(task:str) -> BaseAgent:
    """Transfer control to the weather bot
    Args:
        task (str, optional): Give the weather bot the task. 
    """
    return weather_bot

def transfer_to_orchestrator(task:str) -> BaseAgent:
    """Transfer control back to the ochestrator when assigned tasks are done or need additional supports
    Args:
        task (str, optional): Update the ochestrator what have been done and what errors have been made. 
    """
    return ochestrator


# Initialize the weather bot
weather_bot = AnthropicAgent(
    api_key = os.getenv('ANTHROPIC_API_KEY'),
    name="WeatherBot",
    instructions="""
    You are the weather bot. Your role is to:
    1. Get weather information for specified locations
    2. After completing the task, return control to orchestrator
    3. DO NOT suggest additional weather checks unless asked by the user
    """,
    functions=[get_weather, transfer_to_orchestrator],
)

# Initialize the orchestrator
orchestrator = AnthropicAgent(
    api_key = os.getenv('ANTHROPIC_API_KEY'),
    is_orchestrator = True,
    name="OrchestratorBot",
    instructions="""
    You are the orchestrator bot. Your role is to:
    1. Transfer control to other bots based on user input:
        - Analyze the request and determine which bot should handle it
        - Use transfer functions as follows:
            - Example: transfer_to_agentx(task = "Get temperature in New York")
    
    2. When a taks is done by an agent, it will be transfered to you for you to decide which agent to transfer to for the next task
    """,
    functions=[transfer_to_weather],
)

In [65]:
# Usage of the next
func_name = 'get_weather'
next((f for f in [get_weather, transfer_to_orchestrator] if f.__name__ == func_name),None)

<function smart_team.agents.agent_functions.get_weather(city: str) -> str>

In [72]:
active_agent = orchestrator
cross_agent_memories = []
while True:
    messages = []
    user_input = input("\nEnter your request (or 'exit' to quit): ")
    if user_input.lower() == "exit":
        break
    messages = [{'role': 'user', 'content': user_input + active_agent.instructions}]
    if cross_agent_memories and active_agent.is_ochestrator == True:
        messages.append({'role': 'assistant', 'content': f"The following is the hitorical conversation history: {str(cross_agent_memories)}"})

    result = active_agent.send_message(messages)
    print(result.text)
    cross_agent_memories.append({"role": "user", "content": user_input})
    cross_agent_memories.append({"role": "assistant", "content": result.text})
  
    while result.function_calls:
        for func_call in result.function_calls:
            func_name = func_call["name"]
            func_params = func_call["parameters"]
            cross_agent_memories.append({"role": "assistant", "content": f"Start Function Call: {func_name}({func_params})"})
            ################# Start Transfer Logic #################
            if func_name.startswith("transfer_to_"):
                print(f"Transferring to {func_name}!!!")
                func = next(
                            (f for f in active_agent.functions if f.__name__ == func_name),
                            None
                        )
                new_agent = func(**func_params)
                active_agent = new_agent
                print(f'New Agent Name:{active_agent.name}')
                cross_agent_memories.append({"role": "assistant", "content": f"Transfer Function Call: {func_name}({func_params})"})
                # New Agent Function Execution After Transfer
                messages = [{'role': 'user', 'content': str(cross_agent_memories[-1]) + active_agent.instructions}]
                result = active_agent.send_message(messages)
                print(result.text)
                agent_function_mapping = {fun .__name__: fun for fun in active_agent.functions}
                if active_agent.is_ochestrator is not True:
                    for func_call in result.function_calls:
                        func_name = func_call["name"]
                        func_params = func_call["parameters"]
                        func = agent_function_mapping[func_name]
                        func_result = func(**func_call["parameters"])
                        cross_agent_memories.append({"role": "assistant", "content": f"End Function Call: {func_name}({func_params})"})
                        print(f'Function Result:{func_name} Finished. Result: {func_result}')

        

I'll help transfer this weather-related query to the weather bot.
Transferring to transfer_to_weather!!!
New Agent Name:WeatherBot
I'll help get the weather information for Tokyo and then transfer control back to the orchestrator.

Let me check the temperature in Tokyo:
Function Result:get_weather Finished. Result: Temperature in Tokyo: +11°C


KeyboardInterrupt: 